# 01 • Harmonic Oscillator

Consider a the Hamiltonian harmonic oscillator in one dimension:

$$H(q, p) = T(p) + V(q) = \frac{p^2}{2m} + \frac{k}{2} (q-q_0)^2$$

- $T(p)$ and $V(q)$ are the kinetic and potential energies,
- $q$ and $p$ are the generalized coordinate and momentum,
- $m$ is the mass, $k$ the spring constant, and $q_0$ the equilibrium position.

The `hamiltonian` module provides a base class `Hamiltonian` and a derived class `HarmonicOscillator`, which implements the above system.

In [ ]:
import matplotlib.pyplot as plt
import torch

from hamiltonian import HarmonicOscillator

## Visualize kinetic and potential energies for various parameters

- How does the mass $m$ impact the kinetic energy $T(p)$ for $-1 \leq p \leq 1$?
- How does the spring constant $k$ impact the potential energy $V(q)$ for $-1 \leq q \leq 1$?
- How does the equilibrium position $q_0$ impact the potential energy $V(q)$ for $-1 \leq q \leq 1$?

In [ ]:
q = torch.linspace(-1, 1, 100)
p = torch.linspace(-1, 1, 100)

fig, axes = plt.subplots(ncols=3, figsize=(12, 3))

for mass in (0.1, 0.5, 1):
    H = HarmonicOscillator(mass=mass)
    T = H.ekin(p)
    axes[0].plot(p, T, label=f"{mass=:}")
    axes[0].set_xlabel("p / a.u.")
    axes[0].set_ylabel("T / a.u.")

for spring_constant in (0.1, 1, 10):
    H = HarmonicOscillator(spring_constant=spring_constant)
    V = H.epot(q)
    axes[1].plot(q, V, label=f"{spring_constant=:}")
    axes[1].set_xlabel("q / a.u.")
    axes[1].set_ylabel("V / a.u.")

for equilibrium_position in (-0.5, 0, 0.5):
    H = HarmonicOscillator(equilibrium_position=equilibrium_position)
    V = H.epot(q)
    axes[2].plot(q, V, label=f"{equilibrium_position=:}")
    axes[2].set_xlabel("q / a.u.")
    axes[2].set_ylabel("V / a.u.")

for ax in axes.flat:
    ax.legend()
fig.tight_layout()

## Simulate the time evolution

The base class `Hamiltonian` implements the canonical equations of motion,

$$\frac{\text{d}q}{\text{d}t} = \frac{\partial H}{\partial p} \quad \text{and} \quad \frac{\text{d}p}{\text{d}t} = -\frac{\partial H}{\partial q},$$

via PyTorch's `autograd` mechanism. This gives access to the above derivatives, which are used in the `Hamiltonian.step()` method, a kick-drift symplectic Euler integrator.

The derived class `HarmonicOscillator` inherits this functionality and we can use the `step` method to simulate how the generalized coordinate $q(t)$ and momentum $p(t)$ evolve over time.

In [ ]:
q = torch.tensor(1.0)  # initial coordinate
p = torch.tensor(0.0)  # initial momentum

steps = 1000  # make 1000 steps
delta_t = 0.01  # time step

H = HarmonicOscillator()

qt, pt = [q.item()], [p.item()]
for step in range(steps):
    q, p = H.step(q, p, delta_t)
    qt.append(q.item())
    pt.append(p.item())
qt = torch.tensor(qt)
pt = torch.tensor(pt)

fig, axes = plt.subplots(ncols=2, figsize=(8, 3))

axes[0].plot(qt, label="q")
axes[0].plot(pt, label="p")
axes[0].set_ylabel("q, p / a.u.")

axes[1].plot(H.epot(qt), label="V(q)")
axes[1].plot(H.ekin(pt), label="T(p)")
axes[1].plot(H(qt, pt), label="H(q,p)")
axes[1].set_ylabel("energy / a.u.")

for ax in axes.flat:
    ax.legend()
    ax.set_xlabel("step")
fig.tight_layout()

## Simulate multiple realizations

The `autograd` mechanism even allows to diferentiate multiple inital conditions at the same time. Below is an example for two different initial conditions. Both versions start at the equilibrium position, but with opposite momenta.

In [ ]:
q = torch.tensor([0.0, 0.0])  # initial coordinates
p = torch.tensor([-1.0, 1.0])  # initial momenta

qt0, qt1 = [q[0].item()], [q[1].item()]
for step in range(steps):
    q, p = H.step(q, p, delta_t)
    qt0.append(q[0].item())
    qt1.append(q[1].item())
qt0 = torch.tensor(qt0)
qt1 = torch.tensor(qt1)

fig, ax = plt.subplots(figsize=(4, 3))
ax.plot(qt0, label="$p_0$ = -1")
ax.plot(qt1, label="$p_0$ = 1")
ax.set_xlabel("step")
ax.set_ylabel("q / a.u.")
ax.legend()
fig.tight_layout()

In the next notebook, we will train a *Neural* Hamiltonian to mimic the behavior of our analytical harmonic oscillator.